In [9]:
#Imports
import cv2
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import image as image
import easygui

In [2]:
#1 Open a user-selected image
#f = easygui.fileopenbox(filetypes=["*.jpg","*.jpeg","*.png"])#open user file select
#I = cv2.imread(f)#set a file to work with
#Original = I.copy()#create a copy incase theres a problem

In [3]:
Img = cv2.imread("Cards.jpg")

In [13]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract'

ImgLetter = cv2.imread("sampleLetter3.png")
ImgText = cv2.imread("sampleText.png")

#ImgLetterB = cv2.cvtColor(ImgLetter, cv2.COLOR_BGR2GRAY)



print(pytesseract.image_to_string(ImgLetter))
print(pytesseract.image_to_string(ImgText))


sample



In [14]:
image = cv2.imread('sampleLetter3.png',0)
thresh = cv2.threshold(image, 150, 255, cv2.THRESH_BINARY_INV)[1]

result = cv2.GaussianBlur(thresh, (5,5), 0)
result = 255 - result

data = pytesseract.image_to_string(result, lang='eng',config='--psm 6')
print(data)

K



In [46]:
import pytesseract

#Path to your Tesseract-OCR
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract'
#Imput takes small section of top corner of card
image = cv2.imread('sampleLetter.png',0)
#Gets a threshold
thresh = cv2.threshold(image, 150, 255, cv2.THRESH_BINARY_INV)[1]
#Blurs it with threshold
imBlur = cv2.GaussianBlur(thresh, (5,5), 0)
imBlur = 255 - imBlur

cv2.imshow("imBlur", imBlur)
key = cv2.waitKey(0)
#convert to text from section of image
textFromImage = pytesseract.image_to_string(result, lang='eng',config='--psm 6')
print("The card is: ",textFromImage)

The card is:  K



In [7]:
import Image
from tesseract import image_to_string

print(image_to_string(Image.open('test.png')))

NameError: name 'Image' is not defined